In [ ]:
import pandas as pd
import numpy as np

import itertools
import os

In [ ]:
# Ignore 'trade_update_date', 'run_date', 'vessel_name', 'secondary_notify_party_1','container_number'
usecols = ['identifier','shipper_party_name', 'shipper_address','harmonized_number']

In [ ]:
dtype = {'identifier':str,'shipper_party_name':str,'shipper_address':str,'harmonized_number':str}

In [ ]:
Enigma = pd.read_csv('BofL6country.zip',usecols=usecols,dtype=dtype,compression='zip')

In [ ]:
Enigma = Enigma.dropna(subset=['shipper_party_name'])
Enigma = Enigma.reset_index(drop=True)
# Clean Enigma and Camaras and turn them into series
replace_char = ",.=_-><\'\":;()!~"
replace_dict = {key:value for (key,value) in zip(replace_char,itertools.repeat(''))}
Enigma['cl_shipper_party_name'] = Enigma['shipper_party_name'].str.translate(str.maketrans(replace_dict))
Enigma = Enigma.drop_duplicates(subset=['cl_shipper_party_name'])
Enigma = Enigma.reset_index(drop=True)

In [ ]:
def create_slurm_job(shipper_size,n_steps):
    import math
    increment = math.floor(shipper_size/n_steps)
    start_from = 1
    for i in range(n_steps):
        if i == n_steps-1:
            print('srun --nodes=1 --ntasks=1 --exclusive python ~/FEIII-SHIP/Data_Code_Matches/Enigma_Enigma_matching.py {} {} {}.pkl.zip &'.format(start_from,shipper_size-1,i))
        else:
            print('srun --nodes=1 --ntasks=1 --exclusive python ~/FEIII-SHIP/Data_Code_Matches/Enigma_Enigma_matching.py {} {} {}.pkl.zip &'.format(start_from,start_from+increment,i))
        start_from = start_from + increment + 1

In [ ]:
create_slurm_job(len(Enigma),10)

### Result

In [ ]:
path = 'match_by_levenshtein/'
data = pd.concat(pd.read_pickle(path + f,compression='zip') for f in os.listdir(path))

In [ ]:
test['identifier_right'].iloc[2]

In [ ]:
data.reset_index(drop=True).to_csv('Enigma_Enigma_6countries.csv',index=False)

In [ ]:
data = data.reset_index(drop=True)

In [ ]:
data['name_score'].mean()

In [ ]:
data['address_score'].mean()

In [ ]:
data[data['name_score'] >= 0.85]['address_score'].mean()

In [ ]:
data[data['name_score'] >= 0.90]['address_score'].mean()

In [ ]:
data[data['name_score'] >= 0.95]['address_score'].mean()

In [ ]:
data.shape

In [ ]:
data[data['address_score']>=0.6].shape

In [ ]:
data[data['address_score']>=0.6]['name_score'].mean()

In [ ]:
scatter_plot = data.plot.scatter('name_score','address_score',s=0.5)
fig = scatter_plot.get_figure()
fig.savefig("name_address.png",dpi=200)

In [ ]:
data[(data['name_score'] >= 0.95) & (data['address_score']<=0.5)]